<a href="https://colab.research.google.com/github/batistahpedro/Amazon-New-Oil-Frontier/blob/main/Amazon_New_Oil_Frontier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import folium

In [ ]:
import ee
ee.Authenticate()
ee.Initialize(project='ee-batistahpedro')

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Spatial Analysis ##

Shapefiles

In [ ]:
import geopandas as gpd

base_path = "/content/drive/MyDrive/Amazon-New-Oil-Frontier/shapefiles"

# Polygons of major sedimentary basins across Brazil
sedimentary_basins = gpd.read_file(f"{base_path}/sedimentary_basins/bacias_gishub_db.shp")

# Extent of Brazilian coastal and marine ecological systems
coastal_marine_systems = gpd.read_file(f"{base_path}/coastal_marine_systems/Sistema_Costeiro_Marinho.shp")

# Priority areas for conservation in coastal and marine zones (2nd update)
priority_areas_zcm = gpd.read_file(f"{base_path}/priority_areas_zcm/ZCM_2a_atualizacao.shp")

# Priority areas for biodiversity conservation in the Amazon biome (2nd update)
priority_areas_amazon = gpd.read_file(f"{base_path}/priority_areas_amazon/Amazonia_2a_atualizacao.shp")

# Priority conservation areas in the Cerrado and Pantanal biomes (2nd update)
priority_areas_cerrado_pantanal = gpd.read_file(f"{base_path}/priority_areas_cerrado_pantanal/Cerrado_Pantanal_2a_atualizacao.shp")

# Priority conservation areas in the Caatinga biome (2nd update)
priority_areas_caatinga = gpd.read_file(f"{base_path}/priority_areas_caatinga/Caatinga_2a_atualizacao.shp")

# Oil and gas blocks to be offered in the 5th exploration cycle
offered_blocks = gpd.read_file(f"{base_path}/offered_blocks_cycle5/opc_blocos_em_oferta.shp")

# Oil and gas exploration blocks already under contract or licensed
exploration_blocks = gpd.read_file(f"{base_path}/exploration_blocks/block_shp_geoanp.shp")

# Hybrid zones in the Amazon with overlapping conservation and production designations (2nd update)
amazon_hybrid_zones = gpd.read_file(f"{base_path}/amazon_hybrid_zones/Areas_hibridas_2a_atualizacao.shp")


In [ ]:
# List of all geodataframes to check
gdf_list = [
    ("sedimentary_basins", sedimentary_basins),
    ("coastal_marine_systems", coastal_marine_systems),
    ("priority_areas_zcm", priority_areas_zcm),
    ("priority_areas_amazon", priority_areas_amazon),
    ("priority_areas_cerrado_pantanal", priority_areas_cerrado_pantanal),
    ("priority_areas_caatinga", priority_areas_caatinga),
    ("offered_blocks", offered_blocks),
    ("exploration_blocks", exploration_blocks),
    ("amazon_hybrid_zones", amazon_hybrid_zones)
]

# Loop through each and print basic diagnostics
for name, gdf in gdf_list:
    print(f"\n🔎 Checking: {name}")
    print(f"  - Total features: {len(gdf)}")

    # Check CRS
    if gdf.crs:
        print(f"  - CRS: {gdf.crs}")
    else:
        print("  ⚠️ CRS is undefined!")

    # Null or empty geometry
    null_geom = gdf['geometry'].isna().sum()
    empty_geom = gdf['geometry'].is_empty.sum()
    print(f"  - Null geometries: {null_geom}")
    print(f"  - Empty geometries: {empty_geom}")

    # Invalid geometries
    invalid_geom = (~gdf.is_valid).sum()
    print(f"  - Invalid geometries: {invalid_geom}")

    # Columns available
    print(f"  - Columns: {list(gdf.columns)}")



🔎 Checking: sedimentary_basins
  - Total features: 72
  - CRS: EPSG:4326
  - Null geometries: 0
  - Empty geometries: 0
  - Invalid geometries: 7
  - Columns: ['feicao_id', 'situacao', 'objectid', 'name', 'code', 'creation_d', 'modificati', 'camada_id', 'geometry']

🔎 Checking: coastal_marine_systems
  - Total features: 1
  - CRS: EPSG:4674
  - Null geometries: 0
  - Empty geometries: 0
  - Invalid geometries: 1
  - Columns: ['S_COSTEIRO', 'geometry']

🔎 Checking: priority_areas_zcm
  - Total features: 114
  - CRS: EPSG:4674
  - Null geometries: 0
  - Empty geometries: 0
  - Invalid geometries: 2
  - Columns: ['OBJECTID', 'NOME_AP', 'IMP', 'PRIO', 'AcPrinc', 'Ac2', 'Ac3', 'AcPrincNom', 'Area_ha', 'N', 'ID_AP', 'AcPrincDet', 'Ac2Det', 'Ac3Det', 'TemaAcPrin', 'Shape_Leng', 'AcaoPrinc', 'Acao2', 'Acao3', 'geometry']

🔎 Checking: priority_areas_amazon
  - Total features: 860
  - CRS: EPSG:4674
  - Null geometries: 0
  - Empty geometries: 0
  - Invalid geometries: 19
  - Columns: ['OBJECTI

In [ ]:
# ================================================================
# Standardize CRS and fix invalid geometries
# ================================================================
# Ensure all layers use EPSG:4326 and have valid geometries for spatial operations

def fix_geodataframe(gdf, name):
    print(f"\nFixing: {name}")

    if gdf.crs is None or gdf.crs.to_epsg() != 4326:
        print("  - Reprojecting to EPSG:4326")
        gdf = gdf.to_crs(epsg=4326)

    invalid_count = (~gdf.is_valid).sum()
    if invalid_count > 0:
        print(f"  - Repairing {invalid_count} invalid geometries")
        gdf['geometry'] = gdf['geometry'].buffer(0)

    return gdf

coastal_marine_systems = fix_geodataframe(coastal_marine_systems, "coastal_marine_systems")
priority_areas_zcm = fix_geodataframe(priority_areas_zcm, "priority_areas_zcm")
priority_areas_amazon = fix_geodataframe(priority_areas_amazon, "priority_areas_amazon")
priority_areas_cerrado_pantanal = fix_geodataframe(priority_areas_cerrado_pantanal, "priority_areas_cerrado_pantanal")
priority_areas_caatinga = fix_geodataframe(priority_areas_caatinga, "priority_areas_caatinga")
offered_blocks = fix_geodataframe(offered_blocks, "offered_blocks")
sedimentary_basins = fix_geodataframe(sedimentary_basins, "sedimentary_basins")
amazon_hybrid_zones = fix_geodataframe(amazon_hybrid_zones, "amazon_hybrid_zones")



Fixing: coastal_marine_systems
  - Reprojecting to EPSG:4326
  - Repairing 1 invalid geometries

Fixing: priority_areas_zcm
  - Reprojecting to EPSG:4326
  - Repairing 2 invalid geometries

Fixing: priority_areas_amazon
  - Reprojecting to EPSG:4326
  - Repairing 19 invalid geometries

Fixing: priority_areas_cerrado_pantanal
  - Reprojecting to EPSG:4326
  - Repairing 2 invalid geometries

Fixing: priority_areas_caatinga
  - Reprojecting to EPSG:4326
  - Repairing 2 invalid geometries

Fixing: offered_blocks
  - Reprojecting to EPSG:4326
  - Repairing 1 invalid geometries

Fixing: sedimentary_basins
  - Repairing 7 invalid geometries

Fixing: amazon_hybrid_zones
  - Reprojecting to EPSG:4326
  - Repairing 12 invalid geometries


Port + Airport

In [ ]:
port = gpd.GeoDataFrame(
    {
        'name_display': ['Porto de Belém'],
        'name_clean': ['porto_belem'],
        'type': ['port']
    },
    geometry=[Point(-48.49777700, -1,44527800)],
    crs='EPSG:4326'
)

airport = gpd.GeoDataFrame(
    {
        'name_display': ['Aeroporto de Oiapoque'],
        'name_clean': ['aeroporto_oiapoque'],
        'type': ['airport']
    },
    geometry=[Point(-51.79611100, 3.86138900)],
    crs='EPSG:4326'
)



In [ ]:
import folium

# Ensure all datetime columns are converted to string
def sanitize_for_folium(gdf):
    for col in gdf.select_dtypes(include=["datetime64[ns]"]).columns:
        gdf[col] = gdf[col].astype(str)
    return gdf

# Sanitize each GeoDataFrame
sedimentary_basins = sanitize_for_folium(sedimentary_basins)
coastal_marine_systems = sanitize_for_folium(coastal_marine_systems)
priority_areas_amazon = sanitize_for_folium(priority_areas_amazon)
priority_areas_cerrado_pantanal = sanitize_for_folium(priority_areas_cerrado_pantanal)
priority_areas_caatinga = sanitize_for_folium(priority_areas_caatinga)
priority_areas_zcm = sanitize_for_folium(priority_areas_zcm)
offered_blocks = sanitize_for_folium(offered_blocks)
exploration_blocks = sanitize_for_folium(exploration_blocks)
amazon_hybrid_zones = sanitize_for_folium(amazon_hybrid_zones)

# Create base map
m = folium.Map(location=[-10, -55], zoom_start=4, tiles="CartoDB Positron")

# Style helper
def style(color):
    return lambda feature: {
        'color': color,
        'weight': 1,
        'fillOpacity': 0.3
    }

# Add layers as FeatureGroups
layers = [
    ("Sedimentary Basins", sedimentary_basins, "darkred"),
    ("Coastal & Marine Systems", coastal_marine_systems, "darkblue"),
    ("Amazon Priority Areas", priority_areas_amazon, "green"),
    ("Cerrado/Pantanal Priority Areas", priority_areas_cerrado_pantanal, "orange"),
    ("Caatinga Priority Areas", priority_areas_caatinga, "purple"),
    ("ZCM Priority Areas", priority_areas_zcm, "cadetblue"),
    ("Amazon Hybrid Zones", amazon_hybrid_zones, "gray"),
    ("Offered Blocks (5th cycle)", offered_blocks, "black"),
    ("Exploration Blocks", exploration_blocks, "red")
]

for name, gdf, color in layers:
    fg = folium.FeatureGroup(name=name, show=True)
    folium.GeoJson(gdf, style_function=style(color)).add_to(fg)
    fg.add_to(m)

# Add port and airport as separate interactive layers
fg_port = folium.FeatureGroup(name="Port of Belém", show=True)
folium.GeoJson(port, style_function=style("blue")).add_to(fg_port)
fg_port.add_to(m)

fg_airport = folium.FeatureGroup(name="Oiapoque Airport", show=True)
folium.GeoJson(airport, style_function=style("green")).add_to(fg_airport)
fg_airport.add_to(m)

# Add interactive layer control
folium.LayerControl(collapsed=False).add_to(m)

# Display map
m


Priority Areas

In [ ]:
priority_areas = {
    "Amazon": priority_areas_amazon,
    "Cerrado_Pantanal": priority_areas_cerrado_pantanal,
    "Caatinga": priority_areas_caatinga,
    "ZCM": priority_areas_zcm,
    "Hybrid": amazon_hybrid_zones  # newly added
}

for name, gdf in priority_areas.items():
    print(f"\n{name} Priority Areas")
    print(f"- Total features: {len(gdf)}")
    print(f"- Columns: {list(gdf.columns)}")

    # Identify possible columns containing priority classification
    for col in gdf.columns:
        if gdf[col].dtype == 'object':
            unique_vals = gdf[col].dropna().unique()
            if any(keyword in str(unique_vals).lower() for keyword in ["alta", "prior", "prio", "import"]):
                print(f"  Possible priority column: '{col}' → Unique values: {list(unique_vals)}")



Amazon Priority Areas
- Total features: 860
- Columns: ['OBJECTID_1', 'OBJECTID', 'Nome_area', 'COD_area', 'Import_bio', 'Prior_acao', 'Acao_princ', 'Area_ha', 'Acao_Prin', 'Acao_2', 'Acao_3', 'Shape_Leng', 'Shape_Le_1', 'Shape_Area', 'geometry']
  Possible priority column: 'Import_bio' → Unique values: ['Extremamente Alta', 'Alta', 'Muito Alta']
  Possible priority column: 'Prior_acao' → Unique values: ['Extremamente alta', 'Alta', 'Muito alta']
  Possible priority column: 'Acao_Prin' → Unique values: ['Desenvolvimento de turismo de base comunitária sustentável: Adoção de estratégias metodológicas que envolvam comunidades tradicionais para desenvolvimento de turismo de base comunitária', 'Fiscalização e controle de atividades ilegais', 'Criação de UC de Uso Sustentável', 'Recuperação de áreas degradadas', 'Recuperação de áreas degradadas: Melhoria do manejo do solo, água, pastagem', 'Recuperação de áreas degradadas: Restabelecimento de serviços ecossistêmicos (instrumentos de gestão 

In [ ]:
import pandas as pd
import geopandas as gpd

# Define robust cleaning function
def clean_priority(value):
    if pd.isna(value):
        return None

    # Remove line breaks, tabs, extra spaces
    val = str(value).replace("\r", " ").replace("\n", " ").replace("\t", " ").strip()

    # If multiple values, take the first
    for sep in [";", ","]:
        if sep in val:
            val = val.split(sep)[0].strip()

    val = val.lower()

    if "extremamente" in val or "extremely" in val:
        return "Extremely High"
    elif "muito" in val or "very" in val:
        return "Very High"
    elif "alta" in val or "high" in val:
        return "High"
    else:
        return None

# Apply cleaning logic to each dataset individually
priority_areas_amazon["priority_level"] = priority_areas_amazon["Import_bio"].apply(clean_priority)
priority_areas_cerrado_pantanal["priority_level"] = priority_areas_cerrado_pantanal["Import_bio"].apply(clean_priority)
priority_areas_caatinga["priority_level"] = priority_areas_caatinga["Import_bio"].apply(clean_priority)
priority_areas_zcm["priority_level"] = priority_areas_zcm["IMP"].apply(clean_priority)

# Hybrid zones: choose appropriate source column
# Prefer 'Import_bio', fallback to 'IB_pos' if needed
if "Import_bio" in amazon_hybrid_zones.columns:
    source_col = "Import_bio"
elif "IB_pos" in amazon_hybrid_zones.columns:
    source_col = "IB_pos"
else:
    raise ValueError("No valid priority column found in hybrid zones.")

amazon_hybrid_zones["priority_level"] = amazon_hybrid_zones[source_col].apply(clean_priority)

# Confirm cleanup
for name, gdf in {
    "Amazon": priority_areas_amazon,
    "Cerrado_Pantanal": priority_areas_cerrado_pantanal,
    "Caatinga": priority_areas_caatinga,
    "ZCM": priority_areas_zcm,
    "Hybrid": amazon_hybrid_zones
}.items():
    print(f"{name} — unique priority values: {gdf['priority_level'].dropna().unique()}")


Amazon — unique priority values: ['Extremely High' 'High' 'Very High']
Cerrado_Pantanal — unique priority values: ['Very High' 'Extremely High' 'High']
Caatinga — unique priority values: ['Very High' 'High' 'Extremely High']
ZCM — unique priority values: ['High' 'Very High' 'Extremely High']
Hybrid — unique priority values: ['Extremely High' 'Very High' 'High']


In [ ]:
import geopandas as gpd
import pandas as pd

# Concatenate all cleaned layers into a single GeoDataFrame
all_priority_areas = pd.concat(
    [
        priority_areas_amazon[["geometry", "priority_level"]],
        priority_areas_cerrado_pantanal[["geometry", "priority_level"]],
        priority_areas_caatinga[["geometry", "priority_level"]],
        priority_areas_zcm[["geometry", "priority_level"]],
        amazon_hybrid_zones[["geometry", "priority_level"]]
    ],
    ignore_index=True
)

# Keep only the priority levels of interest
all_priority_areas = all_priority_areas[
    all_priority_areas["priority_level"].isin(["Extremely High", "Very High", "High"])
].copy()

# Report
print(f"Total unified features: {len(all_priority_areas)}")
print("Priority levels included:", all_priority_areas["priority_level"].unique())


Total unified features: 1705
Priority levels included: ['Extremely High' 'High' 'Very High']


## Distance to priority areas ##

In [ ]:
# List of all layers to inspect
layers = {
    "exploration_blocks": exploration_blocks,
    "offered_blocks": offered_blocks,
    "port": port,
    "airport": airport
}

# Print structure of each layer
for name, gdf in layers.items():
    print(f"\n📌 Layer: {name}")
    print("Columns:", list(gdf.columns))
    print("First few rows:")
    display(gdf.head(3))



📌 Layer: exploration_blocks
Columns: ['COD_BLOCO', 'COD_FASE_C', 'DAT_ASSINA', 'DAT_TERMIN', 'NOM_BACIA', 'NOM_BLOCO', 'NOM_FANTAS', 'NUM_CONTRA', 'NUM_DESCOB', 'OPERADOR_C', 'RODADA', 'AREA_TOTAL', 'AMBIENTE', 'BLOCOS', 'geometry']
First few rows:


,COD_BLOCO,COD_FASE_C,DAT_ASSINA,DAT_TERMIN,NOM_BACIA,NOM_BLOCO,NOM_FANTAS,NUM_CONTRA,NUM_DESCOB,OPERADOR_C,RODADA,AREA_TOTAL,AMBIENTE,BLOCOS,geometry
0,SO_DE_SGTR,E,31-05-2023,None,Santos,SO_DE_SGTR,SO_Sagitário_OPP1,48610232918202288,0.0,Petróleo Brasileiro S.A. - PETROBRAS,OPP1,1036.0,M,None,"POLYGON ((-44.37544 -25.45833, -44.41927 -25.4..."
1,BUMERANGUE,E,01-06-2023,None,Santos,BUMERANGUE,Bumerangue_OPP1,48610232917202233,0.0,BP Energy do Brasil Ltda.,OPP1,1119.0,M,None,"POLYGON ((-43.53385 -26.5026, -43.53385 -26.49..."
2,ÁGUA-MARIN,E,31-05-2023,None,Campos,ÁGUA-MARIN,Água_Marinha_OPP1,48610232915202244,0.0,Petróleo Brasileiro S.A. - PETROBRAS,OPP1,1300.0,M,None,"POLYGON ((-40.18529 -22.75311, -40.18269 -22.7..."



📌 Layer: offered_blocks
Columns: ['nome_bacia', 'nomenclatu', 'situacao_b', 'nome_setor', 'indice_blo', 'area', 'AreaANP', 'geometry']
First few rows:


,nome_bacia,nomenclatu,situacao_b,nome_setor,indice_blo,area,AreaANP,geometry
0,Santos,S-M-1283,M,SS-AR4,1283,172.673,172.673,"POLYGON ((-47.125 -26.375, -47.125 -26.5, -47...."
1,Santos,S-M-1284,M,SS-AR4,1284,172.673,172.673,"POLYGON ((-47 -26.375, -47 -26.5, -47.125 -26...."
2,Santos,S-M-502,M,SS-AR3,502,174.987,174.987,"POLYGON ((-45.125 -24.75, -45.125 -24.875, -45..."



📌 Layer: port
Columns: ['name_display', 'name_clean', 'type', 'geometry']
First few rows:


,name_display,name_clean,type,geometry
0,Porto de Belém,porto_belem,port,POINT Z (-48.49778 -1 44527800)



📌 Layer: airport
Columns: ['name_display', 'name_clean', 'type', 'geometry']
First few rows:


,name_display,name_clean,type,geometry
0,Aeroporto de Oiapoque,aeroporto_oiapoque,airport,POINT (-51.79611 3.86139)


In [ ]:
import geopandas as gpd
import pandas as pd
import os

# Output directory
output_dir = "/content/drive/MyDrive/Amazon-New-Oil-Frontier/Results"
os.makedirs(output_dir, exist_ok=True)

# Reproject to EPSG:3857 for metric calculations
print("🔄 Reprojecting layers to EPSG:3857...")
priority_areas_3857 = all_priority_areas.to_crs(epsg=3857)
exploration_blocks = exploration_blocks.to_crs(epsg=3857)
offered_blocks = offered_blocks.to_crs(epsg=3857)
port = port.to_crs(epsg=3857)
airport = airport.to_crs(epsg=3857)
print("✅ Reprojection complete.")

# ID columns for identification
id_columns = {
    "exploration_blocks": "COD_BLOCO",
    "offered_blocks": "nomenclatu",
    "port": "Name",
    "airport": "Name"
}

# Function to compute diagnostics
def compute_and_report(gdf, name):
    print(f"\n🟩 Processing: {name}")
    id_col = id_columns.get(name)

    results = []
    for idx, row in gdf.iterrows():
        geom = row.geometry
        feature_id = row[id_col] if id_col in row else f"Feature {idx}"

        intersections = priority_areas_3857[priority_areas_3857.intersects(geom)]

        if not intersections.empty:
            priority_class = ", ".join(intersections["priority_level"].unique())
            intersection_area = intersections.intersection(geom).area.sum()
            pct = (intersection_area / geom.area) * 100 if geom.area > 0 else 0
            dist = 0

            print(f"  ✅ {feature_id} — Intersection found")
            print(f"     Bounds: {geom.bounds}")
            print(f"     Priority level(s): {priority_class}")
            print(f"     Overlap: {pct:.2f}%")
        else:
            priority_class = "No Priority Area"
            pct = 0
            dist = priority_areas_3857.geometry.distance(geom).min()

            print(f"  ❌ {feature_id} — No intersection")
            print(f"     Bounds: {geom.bounds}")
            print(f"     Min. distance to priority area: {dist:.2f} meters")

        results.append((priority_class, pct, dist))

    # Store results
    gdf["priority_class"] = [r[0] for r in results]
    gdf["overlap_pct_priority_area"] = [round(r[1], 2) for r in results]
    gdf["priority_dist"] = [round(r[2] / 1000, 2) for r in results]  # in kilometers
    return gdf

# Run diagnostics and update original GeoDataFrames explicitly
exploration_blocks = compute_and_report(exploration_blocks, "exploration_blocks")
offered_blocks = compute_and_report(offered_blocks, "offered_blocks")
port = compute_and_report(port, "port")
airport = compute_and_report(airport, "airport")

# Export all to Excel
print("\n📤 Exporting results to Excel (in kilometers)...")
exploration_blocks.to_crs(epsg=4326).drop(columns="geometry").to_excel(f"{output_dir}/exploration_blocks_diagnostics_km.xlsx", index=False)
offered_blocks.to_crs(epsg=4326).drop(columns="geometry").to_excel(f"{output_dir}/offered_blocks_diagnostics_km.xlsx", index=False)
port.to_crs(epsg=4326).drop(columns="geometry").to_excel(f"{output_dir}/port_diagnostics_km.xlsx", index=False)
airport.to_crs(epsg=4326).drop(columns="geometry").to_excel(f"{output_dir}/airport_diagnostics_km.xlsx", index=False)
print("✅ All diagnostics completed and exported.")


🔄 Reprojecting layers to EPSG:3857...
✅ Reprojection complete.

🟩 Processing: exploration_blocks
  ❌ SO_DE_SGTR — No intersection
     Bounds: (-4944730.610575172, -2932208.7468948327, -4905063.805779096, -2886623.8977237814)
     Min. distance to priority area: 8222.50 meters
  ✅ BUMERANGUE — Intersection found
     Bounds: (-4854283.524305637, -3094869.4614607133, -4806161.036098136, -3048837.056750045)
     Priority level(s): High
     Overlap: 100.00%
  ✅ ÁGUA-MARIN — Intersection found
     Bounds: (-4487321.1482882425, -2628554.7760136146, -4431617.332699926, -2597159.270458525)
     Priority level(s): Very High
     Overlap: 7.77%
  ❌ PN-T-102A — No intersection
     Bounds: (-4953717.340300674, -669141.0570442461, -4898057.594904037, -613199.6633499706)
     Min. distance to priority area: 27835.18 meters
  ✅ ES-T-517 — Intersection found
     Bounds: (-4431949.763868368, -2199136.228272881, -4424949.759032625, -2187826.687813247)
     Priority level(s): Extremely High
     Ove

Robustness check

In [ ]:
import folium
import geopandas as gpd

# Step 1: Create base map
m = folium.Map(location=[-10, -55], zoom_start=4, tiles="CartoDB Positron")

# Step 2: Reproject all layers to EPSG:4326 for Folium display
priority_display = all_priority_areas.to_crs(epsg=4326)
exploration_display = exploration_blocks.to_crs(epsg=4326)
offered_display = offered_blocks.to_crs(epsg=4326)
port_display = port.to_crs(epsg=4326)
airport_display = airport.to_crs(epsg=4326)

# Step 3: Add base layers
folium.GeoJson(priority_display, name="Priority Areas",
               style_function=lambda x: {"color": "gray", "weight": 1, "fillOpacity": 0.3}).add_to(m)
folium.GeoJson(exploration_display, name="Exploration Blocks",
               style_function=lambda x: {"color": "red", "weight": 2}).add_to(m)
folium.GeoJson(offered_display, name="Offered Blocks",
               style_function=lambda x: {"color": "black", "weight": 2}).add_to(m)
folium.GeoJson(port_display, name="Port",
               style_function=lambda x: {"color": "blue", "weight": 4}).add_to(m)
folium.GeoJson(airport_display, name="Airport",
               style_function=lambda x: {"color": "green", "weight": 4}).add_to(m)

# Step 4: Helper function to plot green buffers from 'priority_dist'
def plot_green_buffer(gdf, label):
    if "priority_dist" not in gdf.columns:
        print(f"⚠️ Skipping {label}: 'priority_dist' column not found.")
        return
    gdf = gdf[gdf["priority_dist"].notna()].copy()
    if gdf.empty:
        print(f"⚠️ No valid distances in {label}.")
        return
    gdf_buffer = gdf.copy()
    gdf_buffer["buffer_geom"] = gdf_buffer.apply(
        lambda row: row.geometry.buffer(row["priority_dist"] * 1000), axis=1  # km to meters
    )
    gdf_buffer = gpd.GeoDataFrame(
        gdf_buffer.drop(columns="geometry"),
        geometry="buffer_geom",
        crs=gdf.crs
    ).to_crs(epsg=4326)

    folium.GeoJson(
        gdf_buffer,
        name=f"{label} Buffer",
        style_function=lambda x: {"color": "green", "weight": 1, "fillOpacity": 0.15}
    ).add_to(m)

# Step 5: Plot all green buffers
plot_green_buffer(exploration_blocks, "Exploration")
plot_green_buffer(offered_blocks, "Offered")
plot_green_buffer(port, "Port")
plot_green_buffer(airport, "Airport")

# Step 6: Layer control and display
folium.LayerControl(collapsed=False).add_to(m)

# Step 7: Display map
m


Statistics

In [ ]:
# Normalize helper
def normalize(text):
    return text.lower().replace("_", " ").strip()

# ----- Offered Blocks -----
foz_offered = offered_blocks[offered_blocks["nome_bacia"].apply(normalize) == "foz do amazonas"]
total_offered = len(foz_offered)
intersecting_offered = foz_offered[foz_offered["priority_class"] != "No Priority Area"]
count_offered = len(intersecting_offered)
percent_offered = (count_offered / total_offered) * 100 if total_offered > 0 else 0

print("📊 Offered Blocks — Foz do Amazonas")
print(f"- Total: {total_offered}")
print(f"- With intersection: {count_offered}")
print(f"- % Intersecting: {percent_offered:.2f}%")

print("\n🔹 Detailed List of Offered Blocks:")
for _, row in foz_offered.iterrows():
    bloco = row["nomenclatu"]
    if row["priority_class"] != "No Priority Area":
        print(f"  ✅ {bloco} — intersects ({row['priority_class']})")
    else:
        print(f"  ❌ {bloco} — no intersection, dist = {row['priority_dist']:.2f} km")

# ----- Exploration Blocks -----
foz_exploration = exploration_blocks[exploration_blocks["NOM_BACIA"].apply(normalize) == "foz do amazonas"]
total_exploration = len(foz_exploration)
intersecting_exploration = foz_exploration[foz_exploration["priority_class"] != "No Priority Area"]
count_exploration = len(intersecting_exploration)
percent_exploration = (count_exploration / total_exploration) * 100 if total_exploration > 0 else 0

print("\n📊 Exploration Blocks — Foz do Amazonas")
print(f"- Total: {total_exploration}")
print(f"- With intersection: {count_exploration}")
print(f"- % Intersecting: {percent_exploration:.2f}%")

print("\n🔹 Detailed List of Exploration Blocks:")
for _, row in foz_exploration.iterrows():
    bloco = row["COD_BLOCO"]
    if row["priority_class"] != "No Priority Area":
        print(f"  ✅ {bloco} — intersects ({row['priority_class']})")
    else:
        print(f"  ❌ {bloco} — no intersection, dist = {row['priority_dist']:.2f} km")



📊 Offered Blocks — Foz do Amazonas
- Total: 47
- With intersection: 27
- % Intersecting: 57.45%

🔹 Detailed List of Offered Blocks:
  ✅ FZA-M-545 — intersects (Extremely High)
  ✅ FZA-M-617 — intersects (Extremely High)
  ✅ FZA-M-407 — intersects (Extremely High)
  ✅ FZA-M-475 — intersects (Extremely High)
  ✅ FZA-M-547 — intersects (Extremely High)
  ❌ FZA-M-619 — no intersection, dist = 8.39 km
  ❌ FZA-M-690 — no intersection, dist = 9.45 km
  ✅ FZA-M-409 — intersects (Extremely High)
  ✅ FZA-M-477 — intersects (Extremely High)
  ✅ FZA-M-1102 — intersects (Extremely High)
  ✅ FZA-M-184 — intersects (Extremely High)
  ❌ FZA-M-255 — no intersection, dist = 3.19 km
  ❌ FZA-M-261 — no intersection, dist = 23.39 km
  ❌ FZA-M-328 — no intersection, dist = 0.30 km
  ❌ FZA-M-263 — no intersection, dist = 18.98 km
  ✅ FZA-M-330 — intersects (Extremely High)
  ❌ FZA-M-265 — no intersection, dist = 18.43 km
  ✅ FZA-M-332 — intersects (Extremely High)
  ❌ FZA-M-267 — no intersection, dist = 20.0